In [67]:
import argparse
import sys

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

max_step=1000
learning_rate=0.001
dropout = 0.9

data_dir='./MNIST_data/'
log_dir='./logs/example2'

In [68]:
mnist = input_data.read_data_sets(data_dir,one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [86]:
tf.reset_default_graph()
sess=tf.InteractiveSession()

In [87]:
with tf.name_scope('input'):
    x_input=tf.placeholder(shape=(None,784),dtype=tf.float32,name='x_input')
    y_input=tf.placeholder(shape=(None,10),dtype=tf.float32,name='y_input')

with tf.name_scope('input_reshape'):
    image_shaped_input=tf.reshape(x_input,shape=(-1,28,28,1))
    '''
    告诉tf，类型是image，名字叫input，数据从image_shaped_input拿，最多显示10个
    '''
    tf.summary.image('input',image_shaped_input,10)

In [88]:
# 初始化参数
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [89]:
def variable_summaries(var):
    print(var.name,var.shape)
    with tf.name_scope('summaries'):
        # 统计参数的均值？ 那么W这样的矩阵，B这样的向量怎么办…
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        
        # 计算参数标准差
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        
        # 记录标准差，最大值，最小值
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        
        # 用直方图记录
        tf.summary.histogram('histogram',var)
    

In [90]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim,output_dim])
            variable_summaries(weights)
        with tf.name_scope('biaes'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        # 没有cnn，就一个dense
        with tf.name_scope('linear_compute'):
            pre_activate=tf.matmul(input_tensor,weights)+biases
            tf.summary.histogram('linear',pre_activate)
        
        activations = act(pre_activate,name='activation')
        tf.summary.histogram('activations',activations)
        return activations

In [91]:
hidden1 = nn_layer(x_input,784,500,'layer1')

layer1/weights/Variable:0 (784, 500)
layer1/biaes/Variable:0 (500,)


In [92]:
with tf.name_scope('dropout'):
    keep_prob=tf.placeholder(dtype=tf.float32)
    tf.summary.scalar('dropout_keep_probability',keep_prob)
    dropped = tf.nn.dropout(hidden1,keep_prob)

In [93]:
y=nn_layer(dropped,500,10,'layer2',act=tf.identity)

layer2/weights/Variable:0 (500, 10)
layer2/biaes/Variable:0 (10,)


In [94]:
# loss
with tf.name_scope('loss'):
    diff = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_input,logits=y)
    with tf.name_scope('total'):
        cross_entropy=tf.reduce_mean(diff)
tf.summary.scalar('loss',cross_entropy)

<tf.Tensor 'loss_1:0' shape=() dtype=string>

In [95]:
# 优化器
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [96]:
# 计算准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_input,1))
    with tf.name_scope('accuracy2'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [104]:
# 合并summary
merged = tf.summary.merge_all()

# 写到磁盘
train_writer = tf.summary.FileWriter(log_dir+'/train',sess.graph)
test_writer = tf.summary.FileWriter(log_dir+'/test')

# 初始化变量
tf.global_variables_initializer().run()

In [105]:
def feed_dict(train):
    if train:
        xs,ys=mnist.train.next_batch(100)
        k=0.9
    else:
        xs,ys=mnist.test.images,mnist.test.labels
        k=1.0
    return {x_input:xs,y_input:ys,keep_prob:k}

In [106]:
for i in range(max_step):
#     batch_x,batch_y=mnist.train.next_batch(50)
    if i % 10 == 0:  # 记录测试集的summary与accuracy
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:  # 记录训练集的summary
        if i % 100 == 99:  # Record execution stats
            # 这两个是干什么用的？
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step],
                                  feed_dict=feed_dict(True),
                                  options=run_options,
                                  run_metadata=run_metadata)
#             train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            print('Adding run metadata for', i)
        else:  # Record a summary
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            train_writer.add_summary(summary, i)
# 请在最后关闭啊…
train_writer.close()
test_writer.close()


Accuracy at step 0: 0.1175
Accuracy at step 10: 0.7179
Accuracy at step 20: 0.8358
Accuracy at step 30: 0.8752
Accuracy at step 40: 0.8884
Accuracy at step 50: 0.8972
Accuracy at step 60: 0.8979
Accuracy at step 70: 0.9059
Accuracy at step 80: 0.9106
Accuracy at step 90: 0.913
Adding run metadata for 99
Accuracy at step 100: 0.9169
Accuracy at step 110: 0.9206
Accuracy at step 120: 0.9156
Accuracy at step 130: 0.9239
Accuracy at step 140: 0.9234
Accuracy at step 150: 0.9293
Accuracy at step 160: 0.9324
Accuracy at step 170: 0.9333
Accuracy at step 180: 0.9318
Accuracy at step 190: 0.9332
Adding run metadata for 199
Accuracy at step 200: 0.9349
Accuracy at step 210: 0.9375
Accuracy at step 220: 0.938
Accuracy at step 230: 0.9425
Accuracy at step 240: 0.9414
Accuracy at step 250: 0.9402
Accuracy at step 260: 0.9441
Accuracy at step 270: 0.9439
Accuracy at step 280: 0.946
Accuracy at step 290: 0.9448
Adding run metadata for 299
Accuracy at step 300: 0.9475
Accuracy at step 310: 0.9447
Acc